In [2]:
from tqdm import tqdm
import sys
import csv
from dateutil.parser import parse
import time
import psycopg2
from psycopg2.extensions import AsIs

maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [3]:
useless = ['', 'X-Folder', 'Message-ID', 'X-Origin', 'Cc', 'Time', 'Attendees', 'Bcc', 'Mime-Version',
            'X-FileName', 'X-bcc', 'X-cc', 'Content-Transfer-Encoding', 'Content-Type', 'Re', 'NameTo', 'NameFrom']
useful = ['Date', 'AddressFrom', 'AddressTo', 'Subject']
print(useful)

#Если что, текста тут и не должно быть - для него нет ключа

['Date', 'AddressFrom', 'AddressTo', 'Subject']


In [4]:
def convert_key(key):
    converter = {
        'Date': 'Date',
        'AddressFrom': 'From',
        'AddressTo': 'To',
        'Subject': 'Subject'
    }
    return converter[key]

In [5]:
def get_info(cur_string, key):
    
    if (key == 'Message'):
        cur_string_copy = (cur_string+'.')[:-1]
        cur_field = cur_string[:cur_string.find('\n')]
        while (cur_field != ''):
            index = cur_string_copy.find('\n')
            cur_field = cur_string_copy[:index]
            cur_string_copy = cur_string_copy[index+1:]
        return cur_string_copy.strip()
    
    else:
        index = cur_string.find('\n'+convert_key(key)+': ')
        if (index == -1):
            return ''
        value_with_trash = cur_string[index+1+len(convert_key(key)+': '):]
        cur_piece = value_with_trash[:value_with_trash.find('\n')+1]
        value_with_trash = value_with_trash[value_with_trash.find('\n')+1:]
        while(value_with_trash[0] == ' ' or value_with_trash[0] == '\t' or value_with_trash[0] == '\n'):
            cur_piece += value_with_trash[:value_with_trash.find('\n')+1]
            value_with_trash = value_with_trash[value_with_trash.find('\n')+1:]
            
        if (key == 'Date'):
            pdt_index = cur_piece.find('(')
            cur_piece = cur_piece[:pdt_index].strip()

        return cur_piece.strip()

In [35]:
def get_string(line):
    norm_dict = dict(line)
    cur_string = norm_dict['message']
    if (cur_string.find('-----Original Message-----') > -1):
        cur_string = cur_string[:cur_string.find('-----Original Message-----')]
    if (cur_string.find('---------------------- Forwarded') > -1):
        cur_string = cur_string[:cur_string.find('---------------------- Forwarded')]
    return cur_string

In [46]:
def create_main_table(file_obj):
    
    conn = psycopg2.connect("dbname='maildata' user='maildiscovery' host='localhost' port='5432' password='mailpass'")
    print('Connected!')
    cur = conn.cursor()
    drop_statement = 'drop table if exists mails'
    create_statement = 'create table mails (Id integer, Date timestamp, AddressFrom text, AddressTo text, Subject text, Message text)'
    cur.execute(drop_statement)
    print('Table dropped!')
    cur.execute(create_statement)
    print('Table created!')
    
    print('Filling started!')
    reader = csv.DictReader(file_obj, delimiter=',')
    with tqdm(total=517401) as pbar:
        cur_max_id = 0
        for line in reader:
            #line is an ordered dictionary
            cur_string = get_string(line)
            letter = dict.fromkeys(useful)
            letter.setdefault('Message')
            letter.setdefault('Id')
            
            for key in letter.keys():
                if (key == 'Id'):
                    letter['Id'] = cur_max_id
                    
                else:
                    letter[key] = get_info(cur_string, key)
                    
            if (letter['AddressTo'] == '' or letter['AddressFrom'] == '' or letter['Message'] == ''):
                pbar.update(1)
                continue
                
            columns = letter.keys()
            values = [letter[column] for column in columns]
            insert_statement = 'insert into mails (%s) values %s'
            cur.execute(insert_statement, (AsIs(','.join(columns)), tuple(values)))
            cur_max_id += 1
            pbar.update(1)
            
    print('Filling finished!')
    conn.commit()
    cur.close()

In [47]:
with open("emails.csv") as f_obj:
    create_main_table(f_obj)

Connected!
Table dropped!


  0%|          | 1/517401 [00:00<19:59:30,  7.19it/s]

Table created!
Filling started!


100%|██████████| 517401/517401 [03:43<00:00, 2312.60it/s]


Filling finished!
